<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/gwbf-notebooks/blob/main/3_GLSB_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Sub basin Buffer

In [16]:
# subbasin
import geopandas as gpd

subbasin_gdf = gpd.read_file('/content/GSLB_subbasin.shp')

print(subbasin_gdf.head())
type(subbasin_gdf)
print(subbasin_gdf.crs)

     HYBAS_ID   NEXT_DOWN   NEXT_SINK    MAIN_BAS  DIST_SINK  DIST_MAIN  \
0  7120591510  7120593250  7120008710  7120008710      969.2      969.2   
1  7120591660  7120593250  7120008710  7120008710      969.3      969.3   
2  7121092250  7121093050  7120008710  7120008710      990.5      990.5   
3  7121091800  7121092250  7120008710  7120008710      997.0      997.0   
4  7120585470  7121091800  7120008710  7120008710     1003.6     1003.6   

   SUB_AREA  UP_AREA       PFAF_ID  ENDO  COAST  ORDER  SORT  \
0     124.3    909.1  772344093100     0      0      3  8593   
1     146.5    146.7  772344092000     0      0      4  8594   
2     155.7    689.3  772344093300     0      0      3  8643   
3     152.7    533.6  772344093400     0      0      3  8650   
4     131.7    269.3  772344095100     0      0      3  8663   

                                            geometry  
0  MULTIPOLYGON (((-114.05437 37.96741, -114.0546...  
1  POLYGON ((-114.03506 37.95776, -114.0353 37.95...  

In [17]:
# gage
import pandas as pd

gage_df = pd.read_csv('/content/gsl_nwm_gage.csv')
print(gage_df.head())
type(gage_df)

         id                                               name  \
0  10011200  WEST FORK BEAR RIVER AT WHITNEY DAM - NR OAKLE...   
1  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE   
2  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...   
3  10015900       SULPHUR CREEK BL RES. - NEAR EVANSTON - WYO.   
4  10016900                        BEAR RIVER AT EVANSTON - WY   

                  River   latitude   longitude  elevation_m    state  
0  WEST FORK BEAR RIVER  40.841614 -110.927119         2797     Utah  
1            BEAR RIVER  40.965225 -110.853508         2432     Utah  
2         SULPHUR CREEK  41.129114 -110.806563         2205  Wyoming  
3         SULPHUR CREEK  41.156058 -110.835176         2173  Wyoming  
4            BEAR RIVER  41.270224 -110.963793         2057  Wyoming  


pandas.core.frame.DataFrame

In [18]:
# well
well_gdf = gpd.read_file('/content/well_shp.shp')
print(well_gdf.head())
type(well_gdf)
print(well_gdf.crs)

        Well_ID          Well_Name    lat_dec    long_dec     GSE  AquiferID  \
0  3.810331e+14  (C-30-18)25aad- 1  38.175796 -113.802750  7098.0          1   
1  3.810371e+14  (C-30-17)30bab- 1  38.176306 -113.795500  7193.0          1   
2  3.811521e+14  (C-30-17)15cab- 1  38.197833 -113.741167  6550.0          1   
3  3.812361e+14  (C-30-18)12cdb- 1  38.210028 -113.815500  7190.0          1   
4  3.821131e+14  (C-28-17)22dda- 1  38.353571 -113.732473  5775.0          1   

  Aquifer_Na State                     geometry  
0  GSL Basin    UT   POINT (-113.80275 38.1758)  
1  GSL Basin    UT   POINT (-113.7955 38.17631)  
2  GSL Basin    UT  POINT (-113.74117 38.19783)  
3  GSL Basin    UT   POINT (-113.8155 38.21003)  
4  GSL Basin    UT  POINT (-113.73247 38.35357)  
EPSG:4326


EPSG:4326


#test

In [36]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from typing import Dict, List, Tuple

def batch_process_gage_well_relationships(
    subbasins_shp: str,
    gage_csv: str,
    wells_csv: str
) -> Tuple[pd.DataFrame, Dict]:
    """
    Process all gages and find their corresponding wells in subbasins

    Returns:
    --------
    Tuple containing:
    - DataFrame with one row per gage-well relationship
    - Dictionary with summary statistics
    """
    # Read all data
    print("Reading input files...")
    subbasins = gpd.read_file(subbasins_shp)
    gages = pd.read_csv(gage_csv)
    wells = pd.read_csv(wells_csv)

    # Create GeoDataFrames
    gages_gdf = gpd.GeoDataFrame(
        gages,
        geometry=[Point(xy) for xy in zip(gages['longitude'], gages['latitude'])],
        crs="EPSG:4326"
    )

    wells_gdf = gpd.GeoDataFrame(
        wells,
        geometry=[Point(xy) for xy in zip(wells['long_dec'], wells['lat_dec'])],
        crs="EPSG:4326"
    )

    # Initialize results
    relationships_data = []
    summary_stats = {
        'total_gages': len(gages),
        'total_wells': len(wells),
        'gages_with_wells': 0,
        'total_relationships': 0,
        'wells_matched': set()
    }

    # Process each gage
    print("Processing gages...")
    for idx, gage in gages_gdf.iterrows():
        # Find containing subbasin for this gage
        containing_subbasin = None
        for _, subbasin in subbasins.iterrows():
            if subbasin.geometry.contains(gage.geometry):
                containing_subbasin = subbasin
                break

        if containing_subbasin is None:
            continue

        # Find wells in this subbasin
        wells_found = False
        for _, well in wells_gdf.iterrows():
            if containing_subbasin.geometry.contains(well.geometry):
                wells_found = True
                summary_stats['wells_matched'].add(well['Well_ID'])

                # Add relationship to results
                relationships_data.append({
                    # Gage information
                    'gage_id': gage['id'],
                    'gage_name': gage['name'],
                    'gage_latitude': gage['latitude'],
                    'gage_longitude': gage['longitude'],
                    'gage_elevation_m': gage['elevation_m'],
                    'river': gage['River'],
                    'gage_state': gage['state'],

                    # Subbasin information
                    'hybas_id': containing_subbasin['HYBAS_ID'],
                    'next_down': containing_subbasin['NEXT_DOWN'],
                    'sub_area': containing_subbasin['SUB_AREA'],
                    'up_area': containing_subbasin['UP_AREA'],
                    'stream_order': containing_subbasin['ORDER'],

                    # Well information
                    'well_id': well['Well_ID'],
                    'well_name': well['Well_Name'],
                    'well_latitude': well['lat_dec'],
                    'well_longitude': well['long_dec'],
                    'well_gse': well['GSE'],
                    'aquifer_id': well['AquiferID'],
                    'aquifer_name': well['Aquifer_Name'],
                    'well_state': well['State']
                })

        if wells_found:
            summary_stats['gages_with_wells'] += 1

    # Create DataFrame from relationships
    relationships_df = pd.DataFrame(relationships_data)

    # Update summary statistics
    summary_stats['total_relationships'] = len(relationships_df)
    summary_stats['wells_matched'] = len(summary_stats['wells_matched'])

    # Print summary
    print("\nProcessing Summary:")
    print(f"Total gages processed: {summary_stats['total_gages']}")
    print(f"Gages with matching wells: {summary_stats['gages_with_wells']}")
    print(f"Total wells matched: {summary_stats['wells_matched']}")
    print(f"Total relationships found: {summary_stats['total_relationships']}")

    return relationships_df, summary_stats

if __name__ == "__main__":
    # Process all relationships
    relationships_df, stats = batch_process_gage_well_relationships(
        'GSLB_subbasin.shp',
        'gsl_nwm_gage.csv',
        'GSLB_1900-2023_wells_with_aquifers.csv'
    )

    # Display first few relationships
    print("\nFirst few gage-well relationships:")
    print(relationships_df.head())

    # Save results
    relationships_df.to_csv('gage_well_relationships.csv', index=False)
    print("\nResults saved to 'gage_well_relationships.csv'")

Reading input files...
Processing gages...

Processing Summary:
Total gages processed: 78
Gages with matching wells: 69
Total wells matched: 2218
Total relationships found: 2663

First few gage-well relationships:
    gage_id                                          gage_name  gage_latitude  \
0  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE      40.965225   
1  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE      40.965225   
2  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   
3  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   
4  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   

   gage_longitude  gage_elevation_m          river gage_state    hybas_id  \
0     -110.853508              2432     BEAR RIVER       Utah  7120514120   
1     -110.853508              2432     BEAR RIVER       Utah  7120514120   
2     -110.806563              2205  SULPHUR CREEK    Wyoming  71205

In [37]:
relationships_df.head()

,gage_id,gage_name,gage_latitude,gage_longitude,gage_elevation_m,river,gage_state,hybas_id,next_down,sub_area,up_area,stream_order,well_id,well_name,well_latitude,well_longitude,well_gse,aquifer_id,aquifer_name,well_state
0,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,40.965225,-110.853508,2432,BEAR RIVER,Utah,7120514120,7120511450,60.6,513.4,1,410154110533201,12-120-12daa01,41.031614,-110.892954,7580.0,1,GSL Basin,WY
1,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,40.965225,-110.853508,2432,BEAR RIVER,Utah,7120514120,7120511450,60.6,513.4,1,410218110541501,12-120-12ba 01,41.038281,-110.904899,7538.0,1,GSL Basin,WY
2,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410341110495002,13-119-34cbb02,41.061336,-110.831286,7533.0,1,GSL Basin,WY
3,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410531110502401,13-119-21ac 01,41.091892,-110.840731,7428.0,1,GSL Basin,WY
4,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410611110473201,13-119-13cb 01,41.103003,-110.792952,7538.0,1,GSL Basin,WY


# 2.Elevation buffer

In [ ]:
import pandas as pd
try:
  final_measurements_delta = pd.read_csv('final_measurements_delta.csv')

           Well_ID        Date       WTE  Delta_WTE
0  381033113480701  2012-09-06  7092.990      0.000
1  381037113474001  2012-09-06  7175.950      0.000
2  381152113442801  1995-11-22  6200.000      0.000
3  381236113485601  2014-07-23  7151.000      0.000
4  382113113435401  2018-05-25  5397.996      2.046


In [ ]:
# Use the 'isin' method to filter final_measurements_delta
filtered_measurements = final_measurements_delta[
    final_measurements_delta['Well_ID'].isin(wells_in_distance_buffer['Well_ID'])
]

print(filtered_measurements.head())

           Well_ID        Date       WTE  Delta_WTE
0  381033113480701  2012-09-06  7092.990      0.000
1  381037113474001  2012-09-06  7175.950      0.000
3  381236113485601  2014-07-23  7151.000      0.000
4  382113113435401  2018-05-25  5397.996      2.046
5  382113113435401  2018-05-24  5397.995      2.045


In [ ]:
def analyze_wte_measurements(wells_in_buffer, filtered_measurements, wte_buffer=10):
    """
    Step 2: Analyze WTE measurements using elevation threshold for all measurements

    Args:
        wells_in_buffer (pd.DataFrame): DataFrame with well information and reach data
        filtered_measurements (pd.DataFrame): DataFrame with WTE measurements and Delta_WTE
        wte_buffer (float): Buffer distance in meters

    Returns:
        pd.DataFrame: DataFrame with selected columns and filtered data
    """
    print("\nStarting WTE Analysis...")

    # 1. Merge all measurements with wells_in_buffer to get reach information
    merged_data = filtered_measurements.merge(
        wells_in_buffer,
        on='Well_ID',
        how='inner'
    )

    # 2. Apply elevation threshold filter to all measurements
    # Convert WTE to meters if needed (multiply by 0.3048)
    elevation_mask = merged_data['WTE'] * 0.3048 > (merged_data['Reach_Elevation'] - wte_buffer)
    filtered_data = merged_data[elevation_mask]

    # 3. Select and reorder required columns
    result = filtered_data[['Reach_ID', 'Reach_Elevation', 'Distance_to_Reach',
                          'Downstream_Gage', 'Well_ID', 'Date', 'WTE', 'Delta_WTE']]

    print("\nWTE Analysis Summary:")
    print(f"Total wells in buffer: {len(wells_in_buffer)}")
    print(f"Number of measurements exceeding threshold: {len(result)}")
    print(f"Number of unique wells in filtered data: {result['Well_ID'].nunique()}")

    return result

In [ ]:
result_df = analyze_wte_measurements(wells_in_distance_buffer, filtered_measurements)


Starting WTE Analysis...

WTE Analysis Summary:
Total wells in buffer: 4002
Number of measurements exceeding threshold: 2696
Number of unique wells in filtered data: 13


In [ ]:
result_df.head()

,Reach_ID,Reach_Elevation,Distance_to_Reach,Downstream_Gage,Well_ID,Date,WTE,Delta_WTE
0,710579638.0,2018.0,15.997118,NaN,381033113480701,2012-09-06,7092.99,0.0
1,710579638.0,2018.0,387.946514,NaN,381037113474001,2012-09-06,7175.95,0.0
2,710258231.0,2033.5,95.717740,NaN,381236113485601,2014-07-23,7151.00,0.0
94047,710537950.0,1691.5,312.526215,NaN,384025114032601,2010-03-17,5520.87,0.0
98891,710549851.0,1668.0,111.400463,NaN,384236114042701,1953-08-03,5518.00,0.0


In [ ]:
result_df.to_csv('result.csv', index=False)